<a href="https://colab.research.google.com/github/MAbuTalha/Research-Paper/blob/main/BertTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# verify GPU availability
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
import torch

print(torch.cuda.current_device())


print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

torch.cuda.empty_cache()

0
1
Tesla K80
True


In [6]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [10]:
import pandas as pd
import numpy as np
# Set notebook mode to work in offline
import warnings
warnings.filterwarnings("ignore")
import re
import re
import string
import sklearn
import itertools


In [14]:
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel


In [18]:
from simpletransformers.classification import ClassificationModel

In [27]:
from google.colab import files
TestData = files.upload()

Saving test.csv to test.csv


In [29]:
for fn in TestData.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(TestData[fn])))

User uploaded file "test.csv" with length 484872 bytes


In [25]:
train_df =pd.read_csv('train.csv')

In [26]:
train_df

,text,label
0,And without justice &amp; rule of law states d...,1
1,For developing countries like Pakistan there a...,1
2,I want to congratulate President Xi Jinping &a...,1
3,Today I visited Nandana Fort where Al Buruni s...,1
4,I congratulate the entire nation &amp; salute ...,1
...,...,...
12395,"Big Match today, https://t.co/PhRQiWPliI",0
12396,"Strategic timeout, sub players ny naswar rakh...",0
12397,"First boundary and Zalmi fans, https://t.co/vm...",0
12398,"Shame on you Shimmy Shimmy,",0


In [30]:
test_df =pd.read_csv('test.csv')

In [31]:
test_df

,text,label
0,Shab-e-Meraj Mubarak to Muslims all over the w...,1
1,"Our Prophet PBUH said ""many nations before you...",1
2,I welcome restoration of the ceasefire along t...,1
3,We also demonstrated to the world Pakistanâ€™s...,1
4,I inaugurated Sri Lanka's High Performance Spo...,1
...,...,...
3030,"Lahore Qalanders, https://t.co/uaYHRJCjSt",0
3031,"Lahore Qalanders is slow poison, Rana Fawad",0
3032,59 plus 59 plus 59 and additional 3 runs requi...,0
3033,Strategic time out mein Lahore Qalanders Dubai...,0


In [32]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

In [33]:
def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

In [34]:
train_df.text=train_df.text.apply(remove_contractions)
test_df.text=test_df.text.apply(remove_contractions)

In [35]:
def clean_dataset(text):
    # Remove hashtag while keeping hashtag text
    text = re.sub(r'#','', text)
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    # Remove tickers
    text = re.sub(r'\$\w*', '', text)
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    # Remove URL, RT, mention(@)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    # Remove words with 4 or fewer letters
    text = re.sub(r'\b\w{1,4}\b', '', text)
    #&, < and >
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    text= ''.join(c for c in text if c <= '\uFFFF') 
    text = text.strip()
    # Remove misspelling words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    # Remove emoji
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    # Remove Mojibake (also extra spaces)
    text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

In [38]:
#!pip install emoji
import emoji 
train_df.text=train_df.text.apply(clean_dataset)
test_df.text=test_df.text.apply(clean_dataset)

In [39]:
X_train_clean= train_df.text
X_test_clean = test_df.text
y_train_clean= train_df.label
y_test_clean = test_df.label

In [40]:
train_df_clean = pd.concat([X_train_clean, y_train_clean], axis=1)
print("Shape of training data set: ", train_df_clean.shape)
print("View of data set: ", train_df_clean.head())

Shape of training data set:  (12400, 2)
View of data set:                                                  text  label
0  without justicerule states disintegrate becaus...      1
1  developing countries Pakistan there lessons le...      1
2  congratulate President Jinpingthe Chinese mome...      1
3  Today visited Nandana where Buruni spent years...      1
4  congratulate entire nationsalute Armed Forces ...      1


In [41]:
eval_df_clean = pd.concat([X_test_clean, y_test_clean], axis=1)
print("Shape of Eval data set: ", eval_df_clean.shape)

Shape of Eval data set:  (3035, 2)


In [42]:
bert_train_args = {
    'learning_rate':5e-5,
    'evaluate_during_training': True,
    'logging_steps': 100,
    'num_train_epochs': 1,
    'evaluate_during_training_steps': 100,
    'save_eval_checkpoints': False,
    'train_batch_size': 32,
    'eval_batch_size': 64,
    'overwrite_output_dir': True,
    'output_dir':'D:/Output-Bert-Test',
    'fp16': False,
    'n_gpu':1,
    'wandb_project': "Bert-Model-Test"
}

In [43]:
model_BertTest = ClassificationModel('bert', 'bert-base-cased', num_labels=2, use_cuda=True, cuda_device=0, args=bert_train_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [44]:
model_BertTest.train_model(train_df_clean, eval_df=eval_df_clean)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


(388,
 {'auprc': [0.9637922121024919,
   0.9723149065305086,
   0.9777347235634581,
   0.9790856989947447],
  'auroc': [0.9567460583026169,
   0.9677961085814757,
   0.9737804660767627,
   0.9754300645846496],
  'eval_loss': [0.2633437099478518,
   0.25683532766561257,
   0.2120604635468529,
   0.2039970917200359],
  'fn': [174, 112, 175, 136],
  'fp': [143, 211, 88, 123],
  'global_step': [100, 200, 300, 388],
  'mcc': [0.7912340512744837,
   0.7883140674342394,
   0.8281902174404301,
   0.8292724196279101],
  'tn': [1334, 1266, 1389, 1354],
  'tp': [1384, 1446, 1383, 1422],
  'train_loss': [0.4956423342227936,
   0.12316792458295822,
   0.10122314095497131,
   0.16261596977710724]})

In [45]:
result, model_outputs, wrong_predictions = model_BertTest.eval_model(eval_df_clean)

Training loss,0.10122
lr,1e-05
global_step,388
_runtime,661
_timestamp,1615968520
_step,6
tp,1422
tn,1354
fp,123
fn,136
mcc,0.82927


Training loss,█▁▁
lr,█▅▁
global_step,▁▁▃▃▆▆█
_runtime,▁▂▃▄▅▆█
_timestamp,▁▂▃▄▅▆█
_step,▁▂▃▅▆▇█
tp,▁█▁▅
tn,▅▁█▆
fp,▄█▁▃
fn,█▁█▄
mcc,▁▁██


In [50]:
from sklearn.metrics import f1_score,accuracy_score,classification_report, confusion_matrix

In [48]:
predictions = []
for x in model_outputs:
    predictions.append(np.argmax(x))

print('f1 score:', f1_score(eval_df_clean['label'], predictions))
print('Accuracy score:', accuracy_score(eval_df_clean['label'], predictions))

f1 score: 0.9165323880116015
Accuracy score: 0.9146622734761121


# **RoBerta**